In [32]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go



In [2]:
# Load data
with open('data-for-publication.xlsx', 'rb') as f:
    dogs = pd.read_excel(f, sheet_name='Dataset', header=0)
display(dogs)

,dog.id,breed_group,sex,sterilization_status,Noise.sensitivity_age,Fearfulness_age,Aggression_age,Surface_age,Separation_age,Impulsivity_age,...,Vas_8_excessive,Vas_9_alwaysplay,Vas_10_complicatedtasks,Vas_11_reacthastily,Vas_12_easilydistracted,Vas_13_cannotwait,impulsive,impulsiveness_disturbes,inattention_score,impulsivity_score
0,1,Jack_Russell_Terrier,female,sterilized,14.241650,14.181420,14.236174,14.233437,14.181420,14.236174,...,2.0,3.0,2.0,2.0,3.0,2.0,0.0,3.0,0.993593,0.097521
1,2,Jack_Russell_Terrier,female,intact,8.693866,8.359294,8.693866,8.353818,NaN,8.422270,...,2.0,3.0,1.0,1.0,2.0,2.0,NaN,2.0,-0.633266,0.904661
2,3,Pinschers_Schnauzers,male,intact,1.992701,1.992701,1.992701,1.992701,1.992701,1.992701,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN
3,4,Pinschers_Schnauzers,male,sterilized,5.405751,5.405751,5.405751,5.405751,5.405751,5.405751,...,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,-0.528441,-0.822966
4,5,German_Shepherd_Dog,male,sterilized,3.875205,4.006568,3.875205,4.006568,4.006568,4.006568,...,3.0,3.0,2.0,2.0,4.0,2.0,1.0,5.0,0.347356,1.438794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15366,15367,Labrador_Retriever,male,sterilized,4.515599,4.515599,4.515599,4.515599,4.515599,4.515599,...,3.0,3.0,1.0,2.0,2.0,2.0,0.0,4.0,0.002578,1.165461
15367,15368,Labrador_Retriever,male,intact,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15368,15369,Pointers,female,intact,5.315693,NaN,5.315693,5.315693,5.315693,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15369,15370,Hunting_terriers,male,intact,1.447332,1.447332,1.447332,1.447332,1.447332,1.447332,...,1.0,3.0,2.0,2.0,3.0,1.0,0.0,1.0,-0.076717,-0.764367


In [ ]:
# for col_name in ['breed_group', 'sex', 'sterilization_status']:
# figure, axes = plt.subplots(1,3, figsize=(20,8))
# sns.barplot(dogs['breed_group'].value_counts().reset_index(), x='breed_group', y='count', ax=axes[0])
fig = px.bar(dogs['breed_group'].value_counts().reset_index(), y='breed_group', x='count', title='Pocty jednotlivych plemien v datach', width=1000, height=600)
fig.show()

In [56]:
df = dogs.groupby(by=['sterilization_status','sex'])['dog.id'].count()#.rename(columns={'dog.id':'count'})#.reset_index().set_index(['sex', 'sterilization_status'])
display(df)
display(df['sterilized', 'male'])

fig = go.Figure(data=[
    go.Bar(name='sterlized', x=['male', 'female'], y=[df['sterilized', 'male'], df['sterilized', 'female']]),
    go.Bar(name='intact', x=['male', 'female'], y=[df['intact', 'male'], df['intact', 'female']])
])
fig.update_layout(barmode='stack', width=700, title='Number of male and female sterilized/intact dogs')
fig.show()

# fig = px.bar(dogs['sex'].value_counts().reset_index(), y='sex', x='count', title='Pocty jednotlivych psov podla poh', width=1000, height=600)
# fig.show()

sterilization_status  sex   
intact                female    6020
                      male      5738
sterilized            female    2114
                      male      1499
Name: dog.id, dtype: int64

np.int64(1499)